In [1]:
import pandas as pd

Importing data and searching data for insights

In [3]:
faults = pd.read_csv("../data/J1939Faults.csv", low_memory = False)
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")
sfc = pd.read_excel("../data/Service Fault Codes_1_0_0_167.xlsx")

C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [4]:
faults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187335 entries, 0 to 1187334
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   RecordID               1187335 non-null  int64  
 1   ESS_Id                 1187335 non-null  int64  
 2   EventTimeStamp         1187335 non-null  object 
 3   eventDescription       1126490 non-null  object 
 4   actionDescription      0 non-null        float64
 5   ecuSoftwareVersion     891285 non-null   object 
 6   ecuSerialNumber        844318 non-null   object 
 7   ecuModel               1122577 non-null  object 
 8   ecuMake                1122577 non-null  object 
 9   ecuSource              1187335 non-null  int64  
 10  spn                    1187335 non-null  int64  
 11  fmi                    1187335 non-null  int64  
 12  active                 1187335 non-null  bool   
 13  activeTransitionCount  1187335 non-null  int64  
 14  faultValue        

Found that 'Condition Exists Engine Protection Torque Derate' is the event that occurs with spn 1569

In [6]:
faults.loc[faults['spn'] == 1569]['eventDescription'].unique()

array(['Condition Exists Engine Protection Torque Derate'], dtype=object)

In [7]:
faults.loc[faults['spn'] == 5246]['eventDescription'].unique()

array([nan], dtype=object)

No event is associated with spn 5246, full derate

In [9]:
fmi = faults.loc[faults['eventDescription'] == 'Condition Exists Engine Protection Torque Derate']['fmi'].unique()
spn = faults.loc[faults['eventDescription'] == 'Condition Exists Engine Protection Torque Derate']['spn'].unique()

print(fmi, spn)

[31] [1569]


In [10]:
faults.loc[faults['eventDescription'] == 'Condition Exists Engine Protection Torque Derate'].count()

RecordID                 10927
ESS_Id                   10927
EventTimeStamp           10927
eventDescription         10927
actionDescription            0
ecuSoftwareVersion       10908
ecuSerialNumber          10665
ecuModel                 10665
ecuMake                  10665
ecuSource                10927
spn                      10927
fmi                      10927
active                   10927
activeTransitionCount    10927
faultValue                   0
EquipmentID              10927
MCTNumber                10927
Latitude                 10927
Longitude                10927
LocationTimeStamp        10927
dtype: int64

10,927 partial derates and 1,195 full derates

In [12]:
(faults['spn'] == 5246).value_counts()

spn
False    1186140
True        1195
Name: count, dtype: int64

In [13]:
faults[faults['faultValue'].isna() == False]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp


In [14]:
pd.DataFrame(faults.loc[faults['spn'] == 5246]['EquipmentID'].value_counts()).reset_index()

,EquipmentID,count
0,1524,72
1,1535,54
2,1525,43
3,1539,32
4,1749,30
...,...,...
210,105360462,1
211,1696,1
212,105411041,1
213,1487,1


In [22]:
faulty_equipment = pd.DataFrame(faults.loc[faults['spn'] == 5246]['EquipmentID']).reset_index()
faulty_array = faulty_equipment['EquipmentID'].unique()

In [52]:
faulty_equipment_df = pd.DataFrame(faulty_array).reset_index()

In [26]:
fault_events = faults.loc[faults['EquipmentID'].isin(faulty_array)]['eventDescription'].unique()

In [28]:
non_fault_events = faults.loc[~faults['eventDescription'].isin(fault_events)]['eventDescription'].unique()

For each equipment that does have a full derate eventually, these are the list of error codes that occur. The list of error codes not associated with any full derate vehicle are also listed.

In [81]:
fault_events

array(['Low (Severity Low) Engine Coolant Level', nan,
       'Abnormal Update Rate Tire Location',
       'Incorrect Data Brake Signal Sensor 2',
       'Special Instructions System Diagnostic Code #1',
       'Special Instructions Wheel Sensor ABS Axle 2 Left',
       'Data May Be Invalid Relative Speed; Rear Axle #1 Left Wheel',
       'Low (Severity Medium) Engine Coolant Level',
       'Low (Severity High) Engine Coolant Level',
       'Condition Exists Cruise Control Enable Switch',
       'Low Voltage (Power Supply (obsolete))',
       'Special Instructions Wheel Sensor ABS Axle 1 Right',
       'Special Instructions Wheel Sensor ABS Axle 1 Left',
       'Not Reporting Data Front Operator Wiper Switch',
       'Incorrect Data Wheel Sensor ABS Axle 1 Left',
       'Data May Be Invalid Relative Speed; Front Axle Left Wheel',
       'Error in System Engine Exhaust Gas Recirculation (EGR) Valve 2 Control',
       'High (Severity Low) Water In Fuel Indicator',
       'Incorrect Data 

In [85]:
non_fault_events

array(['High Current Pressure Modulation Valve ABS Axle 2 Left',
       'High Current Retarder Control Relay',
       'Incorrect Data Warning Light Bulb',
       'High (Severity High) Particulate Matter Trap Monitor',
       'High (Severity High) Transmission Range High Actuator',
       'Abnormal Frequency Wheel Sensor ABS Axle 2 Left',
       'Incorrect Data Engine Ignition Coil #12',
       'High (Severity High) J1939 Network #2',
       'High (Severity High) Cruise Control Enable Switch',
       'Low Voltage (Wheel Sensor ABS Axle 1 Right)',
       'Low (Severity High) Particulate Matter Trap Monitor',
       'High (Severity High) Engine Starter Mode',
       'Data Drifted Low Parking Brake Switch',
       'Low Voltage (Sensor supply voltage 6)',
       'Low Current Engine Turbocharger 1 Compressor Outlet Temperature',
       'Data May Be Invalid Aftertreatment 1 Exhaust Gas Temperature 3',
       'Data May Be Invalid Aftertreatment 1 Exhaust Gas Temperature 1',
       'Data May Be

In [181]:
faults.loc[faults['eventDescription'].isin(fault_events)][['eventDescription', 'spn', 'fmi']].drop_duplicates().dropna()

,eventDescription,spn,fmi
0,Low (Severity Low) Engine Coolant Level,111,17
2,Incorrect Data Steering Wheel Angle,1807,2
8,High (Severity Low) Water In Fuel Indicator,97,15
14,Incorrect Data Brake Signal Sensor 1,1067,2
19,High Voltage (Fuel Level),96,3
...,...,...,...
1125066,High (Severity Medium) Engine Injector Cylinde...,653,16
1144666,Data Drifted Low Commanded Catalyst Reagent Co...,3828,21
1153239,High (Severity Medium) Aftertreatment 1 Intake...,3217,16
1170052,Not Reporting Data Engine Exhaust Gas Recircul...,2791,7


In [34]:
possible_faulty_equipment = faults.loc[faults['eventDescription'].isin(fault_events)]['EquipmentID'].unique()

In [36]:
possible_faulty_df = pd.DataFrame(possible_faulty_equipment).reset_index()

Here is a dataframe containing all vehicles associated with an event that is associated with a vehicle that has spn 5246

In [191]:
partial_derate_df = faults.loc[faults['EquipmentID'].isin(possible_faulty_equipment)].loc[faults['spn'] == 1569]

In [237]:
faults_slim = faults.loc[faults['EquipmentID'].isin(possible_faulty_equipment)]

And here are all vehicles that are associated with spn 5246

In [249]:
faults_slim

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000
1187331,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000
1187332,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000
1187333,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000


In [54]:
faulty_equipment_df

,index,0
0,0,1395
1,1,1630
2,2,1487
3,3,1329
4,4,1419
...,...,...
210,210,2211
211,211,302
212,212,2175
213,213,1854


In [235]:
possible_faulty_df

,index,0
0,0,1439
1,1,1369
2,2,1674
3,3,1417
4,4,1597
...,...,...
1116,1116,2356
1117,1117,105393087
1118,1118,2380
1119,1119,2378


In [303]:
diagnostics_pivot = diagnostics.pivot(index=['Id', 'FaultId'], columns='Name', values='Value')

In [307]:
data = diagnostics_pivot.merge(faults_slim, left_on = 'FaultId', right_on = 'RecordID')

In [225]:
start = faults['EventTimeStamp'].iloc[0]
stop = faults['EventTimeStamp'].iloc[-1]

print(start, "\n", stop)

2015-02-21 10:47:13.000 
 2020-03-06 14:15:34.000


In [173]:
faults['activeTransitionCount'].unique()

array([  2, 127,   1, 126,  76,   5,  13,  22,   3,  14,  15,  36,  59,
        10,  42,   7,   8,  16,   9,   4,  24,  21,  43,   6,  61,  11,
        25,  20,  17,   0,  97,  98,  94,  12,  66,  48,  26,  30,  55,
        56,  23,  54,  18,  57,  58, 113,  32, 105,  68,  99, 100, 101,
       102,  95,  44, 104,  63,  19,  83,  60,  82,  52,  28,  29,  62,
        31, 114, 107,  87, 110,  71,  91, 108,  33,  39,  88,  69,  53,
       106,  64,  70,  92,  93,  27, 109,  65,  96,  67,  84,  40,  34,
        45,  35,  85,  37,  47,  51,  72,  38,  73,  74, 111,  75,  77,
        78,  49,  46, 103,  79,  80,  81,  41,  50, 119,  86, 120, 121,
       122,  89,  90, 112, 115, 123, 124, 116, 125, 117, 118], dtype=int64)

In [177]:
faults['active'].value_counts()

active
True     608454
False    578881
Name: count, dtype: int64

In [320]:
X = data.loc[data['spn'] != 5246]

y = data.loc[data['spn'] == 5246]['spn']

In [324]:
data

,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,...,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,...,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.74375,NaN,...,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
4,NaN,NaN,NaN,NaN,NaN,NaN,11,NaN,NaN,NaN,...,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12821423,NaN,NaN,NaN,NaN,NaN,181.4,NaN,NaN,NaN,NaN,...,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000
12821424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000
12821425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000
12821426,NaN,NaN,NaN,NaN,28606.65625,NaN,NaN,NaN,NaN,NaN,...,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000
